In [8]:
import os 
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
cwd = os.getcwd()
files = os.listdir(cwd + "\\HL7")
files

['charges_0.hl7',
 'charges_1.hl7',
 'charges_2.hl7',
 'charges_3.hl7',
 'charges_4.hl7']

In [9]:
parsed_data = []
for file in files:
    lines = open(cwd + "\\HL7\\" + file, "r").readlines()
    parsed_data.extend([l.split('|') for l in lines if l[0:3] == "PID"])
parsed_data[0]

['PID',
 '1',
 '',
 '789101^^^StJohns^MR',
 '',
 'Smith^Jane^B^^Ms.',
 '',
 '19920322',
 'F',
 '',
 '',
 '456 Oak St^^Gotham^NY^10002^USA',
 '',
 '(555)555-7777',
 '(555)555-7778',
 '',
 'D',
 'C',
 '222-22-2222',
 '876543^PC^NY',
 'English',
 'F',
 'Caucasian',
 'B',
 'Dr. Adams^PrimaryCare^MD',
 '123456',
 'P',
 'N',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'N\n']

In [10]:
from datetime import datetime
from Task.models import Patient

assigned_data = []
for values in parsed_data:
    mrn = values[3].split('^^^')[0] if values[3] else None
    full_name = values[5].split('^') if values[5] else None
    f_name = full_name[1] if full_name[1] else None
    l_name = full_name[0] if full_name[0] else None
    dob = datetime.strptime(values[7], '%Y%m%d') if values[7] else None
    try:
        physician = values[24].split('^')[0] if values[24] else None
    except IndexError:
        physician = None
    
    phone = values[13] if values[13] else None
    address = ', '.join([v for v in values[11].split('^') if v][:-1]) if values[11] else None
    sex = values[8] if values[8] else None
    db_data = {'first_name' : f_name, 'last_name' : l_name, 'dob': dob, 'mrn' : mrn,'physician' : physician,
               'phone' : phone, 'address' : address, 'sex' : sex}
    
    assigned_data.append(db_data.copy())
    patient = Patient()
    patient.first_name = f_name
    patient.last_name = l_name
    patient.dob = dob
    patient.mrn = mrn
    patient.physician = physician
    patient.phone = phone
    patient.address = address
    patient.sex = sex

    patient.save()
Patient.objects.all()

<QuerySet [<Patient: Jane Smith>, <Patient: None None>, <Patient: Olivia Brown>, <Patient: Emily White>, <Patient: James Wilson>, <Patient: Sophia Garcia>, <Patient: Daniel Martinez>, <Patient: Chloe Lee>, <Patient: Ethan Kim>, <Patient: Isabella Rodriguez>, <Patient: Lucas Johnson>, <Patient: Mia Thompson>, <Patient: John Doe>, <Patient: Alice Smith>, <Patient: David Brown>, <Patient: Sarah Wilson>, <Patient: Miguel Garcia>, <Patient: Elena Fernandez>, <Patient: Andrew Lee>, <Patient: Sophia Robinson>, '...(remaining elements truncated)...']>